In [1]:
print('hi there')

hi there


In [4]:
%pip install pandas numpy

928.92s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 11.1 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 12.1 MB/s  0:00:00 eta 0:00:01
Using cached tzdata-2025.3-py2.py3-none-any.whl (348 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np

In [6]:
# 1) Load
df = pd.read_csv("/Users/asanga/suncorp/motor_repair_costs.csv")
print("Shape:", df.shape)
display(df.head())

Shape: (50000, 21)


,Claim_ID,Claim_Date,Vehicle_Make,Vehicle_Model,Year_of_Manufacture,Vehicle_Age,Odometer_km,Loss_Cause,Repair_Shop_ID,Region,...,Parts_Cost,Paint_Cost,Shop_Rate_per_Hour,Total_Repair_Cost,Shop_Repair_Type,FNOL_Notes,Repair_Shop_Report,Photos_Available,Initial_Estimate,Liability_Assessment
0,CLM000001,16/02/2025,Kia,Corolla,2011,14,155321,Side Impact,Shop_0062,West,...,2079.04,752.63,40.25,3153.67,NON_PANEL_REP,Accident occurred when another vehicle crossed...,"Evaluator’s notes: door, taillight show wear f...",True,2552.33,Third Party Responsible
1,CLM000002,6/04/2025,Hyundai,C-Class,2017,8,116957,Single Vehicle Crash,Shop_0319,North,...,1336.58,383.57,88.93,2484.95,PANEL_REC_REP,Crash occurred when vehicle struck an object a...,"Damage identified on roof, mirror, hood, wheel...",True,2928.78,Insured Responsible
2,CLM000003,8/08/2024,Toyota,Altima,2010,15,140811,Head-on Collision,Shop_1699,North,...,2616.45,119.03,75.94,3487.29,PANEL_REC_REP,"Severe head-on crash at Jones Point, impact no...","Inspection notes moderate impact on trunk, fen...",False,3447.80,Third Party Responsible
3,CLM000004,7/12/2023,Toyota,Elantra,2005,20,206379,Rear-end Collision,Shop_0430,Central,...,3265.65,443.56,68.01,3899.64,PANEL_REC_REP,Collision occurred when traffic slowed abruptl...,"Evaluator observed damage to grille, rear bump...",False,4098.17,Insured Responsible
4,CLM000005,11/12/2023,Hyundai,Focus,2007,18,242392,Side Impact,Shop_1590,East,...,1922.22,371.14,56.99,2760.68,NON_PANEL_REP,Side impact occurred at Arias Ports when anoth...,"Assessor confirms that wheel, mirror are misal...",True,2248.64,Third Party Responsible


In [7]:
# 2) Standardise column names (optional but recommended)
#    Comment out if you want to preserve original column names exactly
df.columns = [c.strip() for c in df.columns]

# 3) Basic schema + missing values
print("\nDTypes:\n", df.dtypes)
missing = df.isna().sum().sort_values(ascending=False)
print("\nMissing values (top):\n", missing[missing > 0].head(20))


DTypes:
 Claim_ID                 object
Claim_Date               object
Vehicle_Make             object
Vehicle_Model            object
Year_of_Manufacture       int64
Vehicle_Age               int64
Odometer_km               int64
Loss_Cause               object
Repair_Shop_ID           object
Region                   object
Labour_Hours            float64
Parts_Cost              float64
Paint_Cost              float64
Shop_Rate_per_Hour      float64
Total_Repair_Cost       float64
Shop_Repair_Type         object
FNOL_Notes               object
Repair_Shop_Report       object
Photos_Available           bool
Initial_Estimate        float64
Liability_Assessment     object
dtype: object

Missing values (top):
 Series([], dtype: int64)


In [9]:
# 4) Check duplicates
# 4a) Duplicate Claim_ID
if "Claim_ID" in df.columns:
    dup_claim_ids = df["Claim_ID"].duplicated().sum()
    print("\nDuplicate Claim_ID count:", dup_claim_ids)
else:
    print("\nClaim_ID column not found.")

# 4b) Fully duplicated rows
dup_rows = df.duplicated().sum()
print("Fully duplicated rows:", dup_rows)


Duplicate Claim_ID count: 0
Fully duplicated rows: 0


In [10]:
# 5) Parse dates
# NOTE: Your file uses day-first format like 16/02/2025
if "Claim_Date" in df.columns:
    df["Claim_Date"] = pd.to_datetime(df["Claim_Date"], dayfirst=True, errors="coerce")
    print("\nClaim_Date parsing done. Null dates:", df["Claim_Date"].isna().sum())


Claim_Date parsing done. Null dates: 0


In [11]:
# 6) Ensure numeric columns are numeric (coerce errors to NaN)
num_cols = [
    "Year_of_Manufacture", "Vehicle_Age", "Odometer_km",
    "Labour_Hours", "Parts_Cost", "Paint_Cost", "Shop_Rate_per_Hour",
    "Total_Repair_Cost", "Initial_Estimate", "Settlement_Time_Days"
]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

In [13]:
df.dtypes

Claim_ID                        object
Claim_Date              datetime64[ns]
Vehicle_Make                    object
Vehicle_Model                   object
Year_of_Manufacture              int64
Vehicle_Age                      int64
Odometer_km                      int64
Loss_Cause                      object
Repair_Shop_ID                  object
Region                          object
Labour_Hours                   float64
Parts_Cost                     float64
Paint_Cost                     float64
Shop_Rate_per_Hour             float64
Total_Repair_Cost              float64
Shop_Repair_Type                object
FNOL_Notes                      object
Repair_Shop_Report              object
Photos_Available                  bool
Initial_Estimate               float64
Liability_Assessment            object
dtype: object

In [14]:
# 7) Range / validity checks (domain sanity checks)
issues = {}

def count_invalid(mask, label):
    n = int(mask.sum())
    issues[label] = n
    return n

# Non-negative costs
for c in ["Parts_Cost", "Paint_Cost", "Total_Repair_Cost", "Initial_Estimate"]:
    if c in df.columns:
        count_invalid(df[c] < 0, f"{c} < 0")

# Positive rates/hours
for c in ["Labour_Hours", "Shop_Rate_per_Hour"]:
    if c in df.columns:
        count_invalid(df[c] <= 0, f"{c} <= 0")

# Vehicle sanity
if "Year_of_Manufacture" in df.columns:
    count_invalid((df["Year_of_Manufacture"] < 1980) | (df["Year_of_Manufacture"] > 2026),
                  "Year_of_Manufacture outside [1980, 2026]")

if "Vehicle_Age" in df.columns:
    count_invalid((df["Vehicle_Age"] < 0) | (df["Vehicle_Age"] > 60),
                  "Vehicle_Age outside [0, 60]")

if "Odometer_km" in df.columns:
    count_invalid((df["Odometer_km"] < 0) | (df["Odometer_km"] > 1_000_000),
                  "Odometer_km outside [0, 1,000,000]")

# Cost consistency: Total should be >= (Parts + Paint) (and usually + labour*rate)
# We'll check a conservative rule and a stronger one.
if set(["Total_Repair_Cost", "Parts_Cost", "Paint_Cost"]).issubset(df.columns):
    parts_paint = df["Parts_Cost"].fillna(0) + df["Paint_Cost"].fillna(0)
    count_invalid(df["Total_Repair_Cost"] < parts_paint, "Total_Repair_Cost < Parts_Cost + Paint_Cost")

if set(["Total_Repair_Cost", "Parts_Cost", "Paint_Cost", "Labour_Hours", "Shop_Rate_per_Hour"]).issubset(df.columns):
    est_total = (df["Parts_Cost"].fillna(0) +
                 df["Paint_Cost"].fillna(0) +
                 df["Labour_Hours"].fillna(0) * df["Shop_Rate_per_Hour"].fillna(0))
    # allow a small tolerance in case totals include discounts/taxes/etc.
    count_invalid(df["Total_Repair_Cost"] + 1e-6 < est_total * 0.95, "Total_Repair_Cost suspiciously low vs components")

# Report issues summary
print("\n--- Validity Check Summary ---")
for k, v in issues.items():
    if v > 0:
        print(f"{k}: {v}")


--- Validity Check Summary ---
Labour_Hours <= 0: 41
Total_Repair_Cost < Parts_Cost + Paint_Cost: 3


In [15]:
# 8) Outlier detection (IQR-based) – flag, don't delete
def iqr_bounds(series, k=1.5):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    return q1 - k * iqr, q3 + k * iqr

outlier_flags = pd.DataFrame(index=df.index)

for c in ["Total_Repair_Cost", "Parts_Cost", "Labour_Hours", "Shop_Rate_per_Hour", "Paint_Cost"]:
    if c in df.columns:
        lo, hi = iqr_bounds(df[c].dropna())
        outlier_flags[f"{c}_outlier_iqr"] = (df[c] < lo) | (df[c] > hi)
        print(f"\n{c}: IQR bounds=({lo:.2f}, {hi:.2f}), outliers={int(outlier_flags[f'{c}_outlier_iqr'].sum())}")

# Add an overall "any outlier" flag
outlier_flags["any_outlier_iqr"] = outlier_flags.any(axis=1)
df["any_outlier_iqr"] = outlier_flags["any_outlier_iqr"]

print("\nTotal rows flagged with ANY IQR outlier:", int(df["any_outlier_iqr"].sum()))




Total_Repair_Cost: IQR bounds=(-24.42, 5773.65), outliers=189

Parts_Cost: IQR bounds=(-875.34, 4484.51), outliers=172

Labour_Hours: IQR bounds=(-1.00, 15.00), outliers=181

Shop_Rate_per_Hour: IQR bounds=(-20.39, 180.28), outliers=0

Paint_Cost: IQR bounds=(-171.48, 1173.57), outliers=177

Total rows flagged with ANY IQR outlier: 615


In [16]:
# 9) High-cardinality categorical check
cat_cols = ["Repair_Shop_ID", "Vehicle_Model", "Vehicle_Make", "Region", "Loss_Cause", "Main_Damaged_Part", "Liability_Assessment"]
print("\n--- Cardinality (unique counts) ---")
for c in cat_cols:
    if c in df.columns:
        print(f"{c}: {df[c].nunique()}")

#


--- Cardinality (unique counts) ---
Repair_Shop_ID: 2000
Vehicle_Model: 10
Vehicle_Make: 10
Region: 5
Loss_Cause: 6
Liability_Assessment: 3


In [17]:
#10) Text field readiness checks
text_cols = ["FNOL_Notes", "Repair_Shop_Report"]
print("\n--- Text field completeness ---")
for c in text_cols:
    if c in df.columns:
        nulls = int(df[c].isna().sum())
        avg_len = df[c].fillna("").str.len().mean()
        print(f"{c}: nulls={nulls}, avg_len={avg_len:.1f}")


--- Text field completeness ---
FNOL_Notes: nulls=0, avg_len=104.0
Repair_Shop_Report: nulls=0, avg_len=99.0


In [18]:
# 11) Create a cleaned copy (minimal transformations)
# - Keeps all rows
# - Dates parsed, numerics coerced, flags added
df_clean = df.copy()

print("\n✅ Cleaning completed. df_clean is ready for EDA/modeling.")
display(df_clean.sample(5))


✅ Cleaning completed. df_clean is ready for EDA/modeling.


,Claim_ID,Claim_Date,Vehicle_Make,Vehicle_Model,Year_of_Manufacture,Vehicle_Age,Odometer_km,Loss_Cause,Repair_Shop_ID,Region,...,Paint_Cost,Shop_Rate_per_Hour,Total_Repair_Cost,Shop_Repair_Type,FNOL_Notes,Repair_Shop_Report,Photos_Available,Initial_Estimate,Liability_Assessment,any_outlier_iqr
1364,CLM001365,2025-05-02,Ford,C-Class,2019,6,129276,Side Impact,Shop_1320,North,...,485.18,72.13,3414.10,NON_PANEL_REP,Accident occurred when another vehicle crossed...,"Report highlights issues with roof, hood, with...",True,3118.99,Insured Responsible,False
21593,CLM021594,2025-07-14,Hyundai,Altima,1998,27,278938,Rear-end Collision,Shop_1584,East,...,213.18,35.11,2765.88,SPEC_REP,Collision occurred when traffic slowed abruptl...,"Damage identified on wheel, grille, rear bumpe...",False,2068.55,Insured Responsible,False
38165,CLM038166,2025-05-15,Hyundai,Corolla,2013,12,158753,Parking Lot Incident,Shop_0284,South,...,520.49,90.29,1203.90,PANEL_REC_REP,Accident occurred while reversing in a parking...,"Assessor confirms that trunk, grille are misal...",False,887.60,Third Party Responsible,False
32363,CLM032364,2024-06-15,Honda,Sportage,2014,11,148885,Side Impact,Shop_1179,East,...,899.17,40.93,2896.70,PANEL_REC_REP,Vehicle struck on the passenger side while tur...,"Assessment concludes hood, mirror sustained he...",True,3451.77,Third Party Responsible,False
26290,CLM026291,2024-02-07,Toyota,CX-5,2012,13,162328,Head-on Collision,Shop_0201,South,...,813.99,102.97,2780.01,NON_PANEL_REP,The vehicle collided head-on with another at W...,"Damage identified on roof, fender, side panel,...",False,2328.50,Unknown,False
